## Check that CustomDataModule works

The reason I want to check this is that the augmented images that i logged through tensorboard were completely off. So i wanted to make sure, that the CustomDataModule class works properly. Either it is the dataloaders not working or the images are used wrong in my model.py

1. Instantiate the CustomDataModule and its .setup and .train_dataloader to be able to extract tensor batches from it
2. Create a data variable containing images as tensors (of shape torch.Size([16, 3, 224, 224]))
3. Convert back to actual image using ToPILImage
4. Display it with PIL

So i manually sample from my Dataset and convert the image back to an actual image, then display it with PIL
If it displays an image you recognise then you're not loading images wrong, you might just be using them wrong in the model


The code below only works if the dataloaders in customDataModule does not normalize the tensor.


# Create 3 different .csv files for train, valid and test (while also randomly splitting the dataset)

In [1]:
from createCSV import createCSV
import config
csv = createCSV(
        base_dir = config.BASE_DIR,
        all_csv = config.ALL_CSV,
        train_csv = config.TRAIN_CSV,
        val_csv = config.VAL_CSV,
        test_csv = config.TEST_CSV,
        label_column = config.LABEL_COLUMN,
        test_size = config.TEST_SIZE,
        seed = config.SEED)
csv.set_all_seeds()
csv.df()

['All', 'Approved', 'NonApproved', 'test_set.csv', 'train_set.csv', 'val_set.csv', 'xlbst.csv']
                                           file_name  label parcel_id
0   22-0223605_F78EFF88702EA742E0530EEE260AEFC6.jpeg      1   0223605
1   22-0223605_F78EFF887030A742E0530EEE260AEFC6.jpeg      1   0223605
2   22-0223605_F78EFF88703AA742E0530EEE260AEFC6.jpeg      1   0223605
28  22-0225160_F78EFF887012A742E0530EEE260AEFC6.jpeg      1   0225160
29  22-0225160_F78EFF887013A742E0530EEE260AEFC6.jpeg      1   0225160
30  22-0225160_F78EFF887014A742E0530EEE260AEFC6.jpeg      1   0225160
31  22-0225160_F78EFF887015A742E0530EEE260AEFC6.jpeg      1   0225160
32  22-0225160_F78EFF887016A742E0530EEE260AEFC6.jpeg      1   0225160
33  22-0225160_F78EFF887017A742E0530EEE260AEFC6.jpeg      1   0225160
34  22-0225160_F78EFF88701AA742E0530EEE260AEFC6.jpeg      1   0225160
35  22-0225160_F78EFF88701BA742E0530EEE260AEFC6.jpeg      1   0225160
36  22-0225160_F78EFF88701CA742E0530EEE260AEFC6.jpeg      1   02

# Create model using NN class (LightningModule), create CustomDataModule (LightningDataModule), set up Trainer, and then fit, validate and test

In [1]:
# Train model

from model import NN
from customDataModule import CustomDataModule
import config
from callbacks import MyPrintingCallback, EarlyStopping
import pytorch_lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch import seed_everything
from pytorchModel import *
import torch

seed_everything(
    42, workers=True
)  # By setting workers=True in seed_everything(), Lightning derives unique seeds across all dataloader workers and processes for torch, numpy and stdlib random number generators. When turned on, it ensures that e.g. data augmentations are not repeated across workers.



if __name__ == "__main__":
    
    pytorch_model = pytorchModel(num_classes=config.NUM_CLASSES)
    
    
    logger = TensorBoardLogger("tb_logs", name="my_model") # tb_logs is the folder, name is the name of the experiment/model
    model = NN(
        model=pytorch_model,
        input_size=config.IN_CHANNELS,
        num_classes=config.NUM_CLASSES,
        learning_rate=config.LEARNING_RATE,
    )  # .to(device)
    dm = CustomDataModule(
        data_dir=config.DATA_DIR,
        train_csv=config.TRAIN_CSV,
        val_csv=config.VAL_CSV,
        test_csv=config.VAL_CSV,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    trainer = pl.Trainer(
        logger=logger, # PyTorch lightning will automatically know what we are logging by looking at our model.py logs
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=config.MIN_EPOCHS,
        max_epochs=config.MAX_EPOCHS,
        deterministic=config.DETERMINISTIC#,
#        callbacks=[MyPrintingCallback(), EarlyStopping(monitor="val_loss")],
    )  # deterministic ensures random seed reproducibility

    trainer.fit(model, dm)  # it will automatically know which dataloader to use

# A general place to start is to set num_workers equal to the number of CPU cores on that machine. You can get the number of CPU cores in python using os.cpu_count(), but note that depending on your batch size, you may overflow RAM memory.


Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name           | Type                | Params
-------------------------------------------------------
0 | model          | pytorchModel        | 878 K 
1 | loss_fn        | CrossEntropyLoss    | 0     
2 | f1_score       | MulticlassF1Score   | 0     
3 | accuracy       | MulticlassAccuracy  | 0     
4 | binaryaccuracy | BinaryAccuracy      | 0     
5 | precision      | MulticlassPrecision | 0     
6 | recall         | MulticlassRecall    | 0     
-------------------------------------------------------
878 K     Trainable params
0         Non-trainable params
878 K     Total params
3.514     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]torch.Size([13, 3, 224, 224])


ValueError: too many values to unpack (expected 3)

In [3]:
trainer.validate(model, datamodule=dm)

Validation DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.95it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    1.3940773010253906     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 1.3940773010253906}]

In [4]:
trainer.test(model, datamodule=dm)

Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  2.91it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    1.3940773010253906     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.3940773010253906}]